In [ ]:
import os, sys
import json
import pandas as pd
import rag
from qdrant_client.http import models

In [2]:
df = pd.read_csv('dataset/firefox_featured.csv')
with open('dataset/firefox_featured.csv_embedding.json', 'r') as f:
    embedding = json.load(f)

In [3]:
rag = rag.RAG()

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.18it/s]


In [12]:
rag.initialize_with_embedding('dataset/firefox_featured.csv', embedding, 0.75)

Uploading 5653 embeddings to Qdrant.
All embeddings have been processed and saved to Qdrant.


In [13]:
def rag_query(rag, comment, bug_id):
    similar_ids=rag.find_similar_bugids(comment,bug_id)
    comments=[]
    for similar_id in similar_ids:
        print(similar_id['bug_id'])
        comments.append(rag.id2comment[int(similar_id['bug_id'].split('-')[-1])])
    return comments

In [14]:
df.loc[1, 'comment']

'STR:\n1. Open this website page URL - http://historia.trojmiasto.pl/Blyskawice-SS-czy-pociag-ekspresowy-Historia-niezwyklej-uliczki-n96358.html\n2. Play the movie on the top of the website page\n3. Go to fullscreen mode and start pausing and playing the movie.\n4. Hang will happen with prompt to force crash plugin.\n\nCrashlog hang report:\nhttps://crash-stats.mozilla.com/report/index/38bcb31d-a099-429e-947b-806c02151122\nhttps://crash-stats.mozilla.com/report/index/5b60fc38-0b3f-45f5-af5e-0374b2151122'

In [ ]:
from tqdm import tqdm
import rag as r
chat_prompts = []
for idx, row in tqdm(df.iterrows()):
    chat = r.prompt_construction(row['comment'], row['bug_id'])
    chat_prompts.append(chat)


100%|██████████| 1/1 [00:00<00:00,  1.56it/s]
40it [00:27,  1.45it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
41it [00:28,  1.45it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
42it [00:28,  1.45it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
43it [00:29,  1.45it/s]
100%|██████████| 1/1 [00:00<00:00,  1.56it/s]
44it [00:30,  1.45it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
45it [00:31,  1.44it/s]
100%|██████████| 1/1 [00:00<00:00,  1.56it/s]
46it [00:31,  1.44it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
47it [00:32,  1.44it/s]
100%|██████████| 1/1 [00:00<00:00,  1.56it/s]
48it [00:33,  1.44it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
49it [00:33,  1.44it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
50it [00:34,  1.44it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
51it [00:35,  1.44it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
52it [00:35,  1.45it/s]
100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
53it [00:36,  1.45it/s]
100%|██████████| 1/

In [ ]:
rag.query(df.loc[5651, 'comment'], 2)

In [ ]:
partition_length = int(len(df) * 0.75)
with open('dataset/firefox_featured_chats_train.json', 'w') as f:
    json.dump(chat_prompts[:partition_length])
with open('dataset/firefox_featured_chats_test.json', 'w') as f:
    json.dump(chat_prompts[partition_length:])

In [11]:
rag.client.delete_collection(rag.collection_name)

True

In [6]:
embedding = rag.embedding_model.get_embeddings([df.loc[0, 'comment']])
embedding=embedding[0]

100%|██████████| 1/1 [00:00<00:00,  1.57it/s]


In [7]:
hits = rag.client.search(
            collection_name=rag.collection_name,
            query_vector=embedding,
            limit=3  # Adjust the limit as needed
        )
hits

[ScoredPoint(id=157, version=0, score=0.9999999740582541, payload={'bug_id': 'dataset/firefox_featured-1226960'}, vector=None),
 ScoredPoint(id=158, version=0, score=0.9999999740582541, payload={'bug_id': 'dataset/firefox_featured-1226960'}, vector=None),
 ScoredPoint(id=159, version=0, score=0.9999999740582541, payload={'bug_id': 'dataset/firefox_featured-1226960'}, vector=None)]

In [17]:
similar_bugs = [hit.payload for hit in hits]
similar_bugs

[{'bug_id': 'dataset/firefox_featured-1226960'},
 {'bug_id': 'dataset/firefox_featured-1226960'},
 {'bug_id': 'dataset/firefox_featured-1226960'}]

In [ ]:
def bugid_to_comment(rag, bug_id):
    

In [18]:
rag.bugid_to_comment(similar_bugs[0]['bug_id'])